In [1]:


import time
import gymnasium
from miniwob.action import ActionTypes
from dom_processing import iterate_dom, dict2html, prepare_t5_input
from inference_points import infer_remote_model, parse_t5_output_action
from cc_net5_tokenizer import CCNeT5Tokenizer
from cosine_similarity import find_closest_embeddings
import torch
import numpy as np
import json


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/gymnasium/envs/registration.py:521: UserWarning: WARN: Overriding environment miniwob/bisect-angle-v1 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/gymnasium/envs/registration.py:521: UserWarning: WARN: Overriding environment miniwob/book-flight-v1 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/gymnasium/envs/registration.py:521: UserWarning: WARN: Overriding environment miniwob/book-flight-nodelay-v1 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/gymnasium/envs/registration.py:521: UserWarning: WARN: Overriding environment miniwob

In [2]:


# We remove all elements not needed in the received html to have it cleaned
def clean_html(html):
    tag_remove = ['script', 'img', 'meta', 'head', '<head', 'html', 'link', 'title', 'line', '!--', '!---']
    parts = html.replace('  ', '').split('\n')
    clean_html = ""
    for p in parts:
        if '<' in p:
            # check for tag
            tag = p.split('<')[1].split('>')[0].replace('/', '').split(' ')[0]
            if not tag in tag_remove:
                #print(tag)
                clean_html += p

    return clean_html

drop_fields = ['children', 'tag', 'onmousemove', 'onclick', 'onscroll', 'ontouchmove', 'ontouchstart', 'ontouchend', 'onkeypress', 'children', 'bgColor', 'width', 'height', 'focused', 'fgColor', 'right', 'top', 'left', 'bottom', 'class', 'classes']

# Converts the html to a string
def html2string(element):
    html_conct = "<" + element['tag']

    # add fields
    for field in element:
        if not field in drop_fields and field != "text":
            html_conct += ' ' + field + '="' + element[field] + '"'
    html_conct += '>'

    # add text
    html_conct += element['text']

    # If the element doesn't match, recursively search its children
    if "children" in element:
        for i, child in enumerate(element["children"]):
            path = html2string(child)
            html_conct += path

    html_conct += '</' + element['tag'] + '>'

    return html_conct

def build_tree(element):
        result = {"tag": element.tag}

        # Adding attributes as key-value pairs to the element
        if element.attrib:
            result.update({k: v for k, v in element.attrib.items()})

        # Adding text
        text = element.text.strip() if element.text else ""
        result['text'] = text

        # Adding child elements as nested dictionaries
        if len(element) > 0:
            result["children"] = [build_tree(child) for child in element]

        return result

from lxml import etree
parser = etree.HTMLParser()
def html2dict(html_str):
    tree = etree.fromstring(html_str, parser)# Starting from the root element and building the tree
    root = tree.getroottree().getroot()
    result = build_tree(root)
    return result

# Find whether <canvas> appears before or after the wrap
def wrapCanvasDist(html):
    canvas = len(html.split('<canvas')[0])
    wrap = len(html.split('id="wrap')[0])
    return wrap < canvas

def find_element_path2(element, attributes):
    # Check if the current element matches the attributes
    if all(attr in element.items() for attr in attributes.items()):
        return [element["tag"]]

    # If the element doesn't match, recursively search its children
    if "children" in element:
        for i, child in enumerate(element["children"]):
            path = find_element_path2(child, attributes)
            if path is not None:
                #print(f'element: {element["tag"]}, i:{i}, path:{path}')
                #return [element["tag"] , f"{i+1}/{len(element['children'])}"] + path
                return [element["tag"] , f"{i+1}"] + path
    # If no match was found, return None
    return None

# Update the mapping dictionary of ref with page changes
def update_ref2_random(old_dict, old_rand2ref, new_dict):
    _remvoved_refs_keys = old_dict.keys() - new_dict.keys()
    added_refs_keys = new_dict.keys() - old_dict.keys() # no use of old refs, better to keep them in old_dict

    # create subset_dictionary
    added_refs = {}
    for added_ref in added_refs_keys:
        if added_ref in old_dict.keys():
            added_refs[added_ref] = old_dict[added_ref]
        else:
            added_refs[added_ref] = new_dict[added_ref]

    # attribute random to new refs:
    added_refs_dict, added_rand2refs = randomize_ref_dict(list(added_refs.keys()))

    # Update the old dictionary with the new mapping
    old_dict.update(added_refs_dict)
    old_rand2ref.update(added_rand2refs)

    # These are the new mappings
    return old_dict, old_rand2ref

import random

# find all refs of the dom dict
def find_all_refs(dom):
    refs = []
    if 'ref' in dom.keys():
        refs.append(dom['ref'])
    if 'children' in dom.keys():
        for child in dom['children']:
            f_ref = find_all_refs(child)
            refs += f_ref
    return list(set(refs))

# provide ref list and return dictionary with randomized values
# also return dict to reconstruct it all
def randomize_ref_dict(refs):
    entries = refs.copy()
    random.shuffle(entries)
    ref_dict = {}
    for i, k in enumerate(refs):
        ref_dict[k] = entries[i]

    ref2rand = ref_dict
    rand2ref = {}
    for k in ref2rand.keys():
        rand2ref[ref2rand[k]] = k
    return ref_dict, rand2ref

# Assigns the randomized value to each ref of the dom dict
def attribute_random_refs_dom(dom, refs):
    if 'ref' in dom.keys():
        dom['ref'] = refs[dom['ref']]
    if 'children' in dom.keys():
        new_children = []
        for child in dom['children']:
            new_child = attribute_random_refs_dom(child, refs)
            new_children.append(new_child)
        dom['children'] = new_children
    return dom

def convert_to_nested_dict(elements):
    # Create a dictionary to store elements by their 'ref' value
    element_dict = {element['ref']: element for element in elements}

    # Iterate through the elements to build the nested structure
    for element in elements:
        parent_ref = element['parent']
        if parent_ref != 0:
            parent_element = element_dict.get(parent_ref)
            if 'children' not in parent_element:
                parent_element['children'] = []
            if 'children' not in element:
                # Empty children field
                element['children'] = []
            parent_element['children'].append(element)

    # Find the root element (element with 'parent' = 0)
    root_element = None
    for element in elements:
        if element['parent'] == 0:
            root_element = element
            break

    return root_element

In [7]:
import requests
import json
import random

# Infer the model to generate a plan
def call_T5_planning(utterance):
    t5_input = 'Devise a plan for the following instruction: ' + utterance
    print(f"{t5_input}")
    t5_output = infer_remote_model(t5_input, t5_url + '/infer_t5')['prediction']
    print(f'received output: {t5_output}')
    return t5_output


def call_T5_hierarchy(action_history, utterance, dom):
    html_str = dict2html(dom)
    t5_input = prepare_t5_input(action_history, utterance, html_str)
    print(f't5_input: {t5_input}')
    t5_output = infer_remote_model(t5_input, t5_url + '/infer_t5')
    parts = t5_output['prediction'].split(',')
    t5_action = parts[0].replace(';', '')
    t5_ref = int(parts[1].replace(' ', '').replace(',', ''))
    t5_keydown = ''.join(parts[2])
    if len(parts)>3:
        done = parts[3].replace(' ', '') == 'done'
    else:
        done = ""

    return t5_action, t5_ref, t5_keydown, done

model_endpoints = 'https://4c1f-92-177-9-116.ngrok-free.app'
t5_url = 'https://a45b-34-141-131-82.ngrok-free.app'
t5_url = model_endpoints

In [13]:

# Define processing steps of a time step for the agent.
# Returns an action, and ref dictionaries over the steps.
# Pass ref dict of previous actions, empty for first step.
def observe_and_act_t5_base(observation, ref_dict, rand2ref, action_history):
    # 1: Reformat Screenshot Shape
    screenshot = observation['screenshot']
    print(f'screenshot shape: {screenshot.shape}')

    # 2: Check Utterance
    utterance = observation['utterance']

    # 3: Process DOM
    # convert elements to nested dict structure
    nested_dict = convert_to_nested_dict(observation['dom_elements'])
    # clean
    print(f'nested_dict: {nested_dict}')
    _, _, new_dom = iterate_dom(nested_dict)
    # randomize references
    ref_dict_new, _rand2ref_new = randomize_ref_dict(find_all_refs(new_dom))
    # update previous dictionaries with new observations
    ref_dict, rand2ref = update_ref2_random(ref_dict, rand2ref, ref_dict_new)
    # apply randomized references to observation DOM
    doms = attribute_random_refs_dom(new_dom, ref_dict)

    # 4: Model call
    # a. T5 call
    # - Format for T5 call
    t5_action, t5_ref, t5_keydown, done = call_T5_hierarchy(action_history, utterance, doms)
    print(f't5_action: {t5_action}, t5_ref: {t5_ref}, t5_keydown: {t5_keydown}')

    if t5_action == 'click':
        action_type = ActionTypes.CLICK_ELEMENT
    else:
        action_type = ActionTypes.TYPE_TEXT

    # Sample and Create Action
    action = env.action_space.sample()     # Template for the action.
    action["action_type"] = env.action_space_config.action_types.index(
      action_type
    )
    # Get the ref number by using the dictionary mapping random to old reference
    try:
        action["ref"] = str(rand2ref[t5_ref])
    except:
        print(f'FAIL SELECTING ACTION: rand2ref: {rand2ref.keys()}, ref_dict: {ref_dict.keys()}')
        #print(doms)
        #action["ref"] = str(rand2ref[list(rand2ref.keys())[0]])
    # Add text if action is type_text
    if action_type == ActionTypes.TYPE_TEXT:
        action['text'] = t5_keydown
    print(f'action to perform: {action}')

    return action, ref_dict, rand2ref

In [21]:
exp_results = {}

In [1]:
range_actions = 10 # total amount of allowed actions
max_experiments = 100
counter = 0

task_name = 'click-checkboxes-transfer'
env = gymnasium.make(f"miniwob/{task_name}")

# If the task uses the Hierarchical model
hierarchical = True

# Set up previous_action with default values
previous_action = torch.zeros(577)


while counter < max_experiments:
    if task_name not in exp_results.keys():
        exp_results[task_name] = {
            'successes': 0,
            'fails': 0,
            'unfinished': 0,
            'other': 0
        }

    plan = []

    #try:

    # Episode Data
    dom_episode = []
    action_history = []
    # Empty ref dictionaries at start of episode:
    ref_dict, rand2ref = {}, {}
    observation, info = env.reset()

    if hierarchical:
        # Infer the plan for the hierarchical task
        plan = call_T5_planning(observation['utterance']).split(';')
        print(f'plan to solve: {plan}')
        range_actions = len(plan)

    for i in range(range_actions):
        print(f'- Step {i}: {plan[i]}')

        if hierarchical:
            observation['utterance'] = plan[i]

        # Process observation and return Action + Ref Dicts
        action, ref_dict, rand2ref = observe_and_act_t5_base(observation, ref_dict, rand2ref, action_history)
        # Perform action on environment
        observation, reward, terminated, truncated, info = env.step(action)


        # Episode is over
        if terminated:
            print(reward, terminated)

            if reward > 0:
                # todo: save successful episode:
                # dom, actions, refs, ref_dict to transform doms and actions
                # rewards etc
                print('Success')
                exp_results[task_name]['successes'] += 1
            else:
                print('Fail')
                exp_results[task_name]['fails'] += 1
            break

        time.sleep(0.15)
    #except Exception as e:
    #    print(f'Error loop: {e}')

    if not terminated:
        exp_results[task_name]['unfinished'] += 1

    counter += 1



env.close()
print(exp_results)

NameError: name 'gymnasium' is not defined

In [2]:
counter

0